In [1]:
from object_detection.builders import model_builder
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
import tensorflow as tf
import grab_screen
import time
from PIL import ImageGrab

Config path and checkpoint path

In [14]:
config_path=r"C:\Users\user\Desktop\object_detect\pre_trained_model\ssd_resnet101_v1_fpn_640x640_coco17_tpu-8\pipeline.config"
checkpt_path=r"C:\Users\user\Desktop\object_detect\pre_trained_model\ckpt-6"

In [15]:
configs = config_util.get_configs_from_pipeline_file(config_path)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)


In [16]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(checkpt_path)).expect_partial()

In [17]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [18]:
import cv2
import numpy as np


In [20]:
while(True):
    image=ImageGrab.grab(bbox =(115,250,1300, 880))
    image_np=np.array(image)
    #original_image=image_np
    image_np =cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)  

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
    detections['num_detections'] = num_detections

    
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    category_index = label_map_util.create_category_index_from_labelmap(r"C:\Users\user\Desktop\object_detect\pre_trained_model\ssd_resnet101_v1_fpn_640x640_coco17_tpu-8\label_map.pbtxt")
    #count number of helmet detected
    count=0
    for i in range(len(detections["detection_classes"])):
        if (detections["detection_classes"][i]==0 and detections["detection_scores"][i]>0.1):
            count=count +1
    cv2.putText(image_np_with_detections,"count="+str(count), (int(15),int(30)), cv2.FONT_HERSHEY_TRIPLEX, 1, (0,0,255), 1)        
    
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=0.1,
                    agnostic_mode=False)

    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (500,300)))
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

    """import matplotlib.pyplot as plt
    %matplotlib inline
    plt.figure(figsize=(10,10))
    plt.imshow(image_np_with_detections)
    plt.show()"""
